In [9]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

In [10]:
# load the dataset
iris = datasets.load_iris()

In [16]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [3]:
# split the dataset into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(iris.data, iris.target, random_state = 1)

In [4]:
clf = DecisionTreeClassifier()
clf.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [5]:
from sklearn.tree import export_graphviz

In [6]:
dot_data = export_graphviz(clf, out_file=None, feature_names=iris.feature_names, class_names = iris.target_names)

In [7]:
import pydotplus
graph = pydotplus.graph_from_dot_data(dot_data)

In [8]:
graph.write_pdf("iris.pdf")

True

In [9]:
Y_train_pred = clf.predict(X_train)

In [10]:
Y_test_pred = clf.predict(X_test)

In [11]:
from sklearn.metrics import confusion_matrix

In [12]:
confusion_matrix(Y_train, Y_train_pred)

array([[37,  0,  0],
       [ 0, 34,  0],
       [ 0,  0, 41]], dtype=int64)

In [13]:
confusion_matrix(Y_test, Y_test_pred)

array([[13,  0,  0],
       [ 0, 15,  1],
       [ 0,  0,  9]], dtype=int64)

In [17]:
def entropy(y):
    # total count of elements in y
    d=y.count()
    #different values of y
    vals=set(y)
    e=0
    for v in vals:
        # no of elements in y with v class
        d_o=y[y==v].count()
        e=e-((d_o/d)*math.log(d_o/d))
    return e
    
    
def gain(df,y,s_f):
    possible_values=set(df[s_f])
    # total no of elements in df
    s=y.count()
    #entropy before split
    entropy_s=entropy(y)
    entropy_s_v=0
    for val in possible_values:
        y_o=y[df[s_f]==val]
        #total no of elements in df with val
        s_v=df[df[s_f]==val][s_f].count()
        e=entropy(y_o)
        entropy_s_v=entropy_s_v+((s_v/s)*e)
    g=entropy_s-entropy_s_v
    return g
    
    
def build_tree(df, y, unused_features):
    global node
    #base case
    # 1. unused is empty
    if(len(unused_features)==0):
        return
    # 2. y contains only one distinct value
    if(len(set(y))==1):
        return
    
    best_feature = ""
    max_gain=-10000
    for f in unused_features:
        g=gain(df,y,f)
        # update best feature so that that particular feature
        # makes least number of mistakes
        if(max_gain<g):
            max_gain=g
            best_feature=f
        
    # here you should know the best feature
    # print it out
    print("Best Feature ", best_feature)
   
    print("Gain Ratio ",max_gain)
    # remove best feature from unused features
    unused_features.remove(best_feature)
    # loop over possible values of best feature
    for val in set(df[best_feature]):
        df_r=df[df[f]==val]
        y_r=y[df[f]==val]
        print("Node",node)
        print("Count of 0 ",y_r[y_r==0].count())
        print("Count of 1 ",y_r[y_r==1].count())
        print("Count of 2 ",y_r[y_r==2].count())
        print("Current Entropy ",entropy(y_r))
        if(entropy(y_r)==0):
            print("Reached leaf")
        else:
            print("Gain Ratio ",max_gain)
        # call build tree recursively
        node+=1
        build_tree(df_r,y_r,unused_features)
    
      
y = pd.DataFrame(iris.target)
y=y[0]
node=1
unused_features = set(iris.feature_names)
print("Node 0")
print("Count of 0 ",y[y==0].count())
print("Count of 1 ",y[y==1].count())
print("Count of 2 ",y[y==2].count())
build_tree(df, y, unused_features)

Node 0
Count of 0  50
Count of 1  50
Count of 2  50


KeyError: 'sepal length (cm)'